In [1]:
import numpy as np
import nimare
from simulation import Foci_generator
from permutation import foci_dataset

200806-16:27:43,815 nipype.utils INFO:
	 Running nipype version 1.4.2 (latest: 1.5.0)


INFO:nipype.utils:Running nipype version 1.4.2 (latest: 1.5.0)


In [2]:
population_center = np.array([[63,67,50], [63,35,50], [31,73,31], [58,74,31],
                            [28,71,50], [28,38,50], [57,44,66], [34,44,66]])
probs = np.array([0.35,0.50,0.10,0.05]) # probabilities of generating 0/1/2/3 foci
mask_file = 'mask.nii'

In [3]:
%%time
# two sample comparison   
n_total1,  n_total2 = 25, 100
n_valid1, n_valid2 = 10, 10

sigma = 10 # results in (Eickhoff et al, 2009)
grid_width = 33

simulation = Foci_generator(population_center, probs, mask_file)

foci_coords1 = simulation.sample_drawer(n_total1, n_valid1, sigma, grid_width)
foci_coords2 = simulation.sample_drawer(n_total2, n_valid2, sigma, grid_width)

dataset1 = foci_dataset(foci_coords1)
dataset2= foci_dataset(foci_coords2)

CPU times: user 1min 16s, sys: 14.8 s, total: 1min 30s
Wall time: 14.9 s


In [4]:
%%time
new_dataset1, new_dataset2 = two_group_permutation(foci_coords1, foci_coords2)

CPU times: user 9.09 ms, sys: 3.49 ms, total: 12.6 ms
Wall time: 2.93 ms


In [7]:
a = foci_dataset(new_dataset1)

In [8]:
def ale_diff(dataset1, dataset2):
    """
    Subtraction of ALE maps for given datasets
    """
    ale1 = nimare.meta.cbma.ALE()
    ale2 = nimare.meta.cbma.ALE()
    
    ale1.fit(dataset1)
    ale2.fit(dataset2)
    
    ale_subtraction = nimare.meta.cbma.ALESubtraction()
    ale_subtraction_fit = ale_subtraction.fit(ale1, ale2)
    
    ale_diff = ale_subtraction_fit.get_map(name='z_desc-group1MinusGroup2', return_type='map')
    return ale_diff

In [9]:
%%time
a = ale_diff(dataset1, dataset2)

CPU times: user 8min 1s, sys: 35.7 s, total: 8min 36s
Wall time: 8min 53s


In [16]:
b = a.reshape(1,228453)

In [17]:
b.shape

(1, 228453)

In [33]:
def ALE_subtraction_pvalue(n_valid1, n_total1, n_valid2, n_total2, sigma = 10, grid_width = 33, N=50):
    simulation = Foci_generator(population_center, probs, mask_file)
    # draw the coordinates of foci
    foci_coords1 = simulation.sample_drawer(n_total1, n_valid1, sigma, grid_width)
    foci_coords2 = simulation.sample_drawer(n_total2, n_valid2, sigma, grid_width)
    # convert numpy array into the form of `nimare.dataset`
    dataset1 = foci_dataset(foci_coords1)
    dataset2= foci_dataset(foci_coords2)
    
    original_ale_diff = ale_diff(dataset1, dataset2)
    
    ALE_subtraction_array = np.empty((0,228453))
    for i in range(N):
        new_foci_coords1, new_foci_coords2 = two_group_permutation(foci_coords1, foci_coords2)
        new_dataset1 = foci_dataset(new_foci_coords1)
        new_dataset2 = foci_dataset(new_foci_coords2)
        new_ale_diff = ale_diff(new_dataset1, new_dataset2)
        ALE_subtraction_array = np.concatenate((ALE_subtraction_array,new_ale_diff), axis=0)
    # obtain np.array of shape (228453, N) through transpose
    ALE_subtraction_array = ALE_subtraction_array.T
    
    # save the results in npy file
    outfile = 'ALE_subtraction_after_permutation.npy'
    np.save(outfile, ALE_subtraction_array)
    return ALE_subtraction_array

In [34]:
n_total1,  n_total2 = 25, 100
n_valid1, n_valid2 = 10, 10

sigma = 10 # results in (Eickhoff et al, 2009)
grid_width = 33

In [13]:
def ALE_subtraction_result(foci_coords1, foci_coords2, N=50):
    
    ALE_subtraction_array = np.empty((0,228453))
    for i in range(N):
        new_foci_coords1, new_foci_coords2 = two_group_permutation(foci_coords1, foci_coords2)
        new_dataset1 = foci_dataset(new_foci_coords1)
        new_dataset2 = foci_dataset(new_foci_coords2)
        new_ale_diff = ale_diff(new_dataset1, new_dataset2)
        ALE_subtraction_array = np.concatenate((ALE_subtraction_array,new_ale_diff), axis=0)
    # obtain np.array of shape (228453, N) through transpose
    ALE_subtraction_array = ALE_subtraction_array.T
    
    # save the results in npy file
    outfile = 'ALE_subtraction_after_permutation.npy'
    np.save(outfile, ALE_subtraction_array)
    return ALE_subtraction_array

In [14]:
a = ALE_subtraction_result(foci_coords1, foci_coords2, N=2)

KeyboardInterrupt: 

In [18]:
a = np.array([1,2,3,4])

In [22]:
a[3:]

array([4])

In [24]:
np.sqrt(3*32**2)

55.42562584220407

In [26]:
a = np.array([1,1,1])
b = np.array([33,33,33])
dist = np.linalg.norm(a-b)

In [27]:
dist

55.42562584220407

In [28]:
5449**(1/3)

17.59701223387403

In [29]:
a = np.array([1,2,3,4])

In [30]:
a[a<3] = 0

In [31]:
a

array([0, 0, 3, 4])

In [32]:
33384+2553

35937

In [33]:
91*109*91

902629

In [34]:
550301+352328

902629

In [43]:
import numpy as geek 
   
array = geek.arange(12).reshape(2, 2, 3) 
print("Original array : \n", array) 
   
# Putting new elements 
a = geek.place(array, array%2==0, [15, 25, 35, 45, 55, 65]) 
print("\nPutting up elements to array: \n", array) 
  
  


Original array : 
 [[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]]

Putting up elements to array: 
 [[[15  1 25]
  [ 3 35  5]]

 [[45  7 55]
  [ 9 65 11]]]


In [40]:
6 % 2

0

In [20]:
import itertools

%time
xx,yy,zz = 91,109,91
coordinate = []
for element in itertools.product(np.arange(1,xx+1), np.arange(1,yy+1), np.arange(1,zz+1)):
    coordinate.append(list(element))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


In [21]:
coordinate

[[1, 1, 1],
 [1, 1, 2],
 [1, 1, 3],
 [1, 1, 4],
 [1, 1, 5],
 [1, 1, 6],
 [1, 1, 7],
 [1, 1, 8],
 [1, 1, 9],
 [1, 1, 10],
 [1, 1, 11],
 [1, 1, 12],
 [1, 1, 13],
 [1, 1, 14],
 [1, 1, 15],
 [1, 1, 16],
 [1, 1, 17],
 [1, 1, 18],
 [1, 1, 19],
 [1, 1, 20],
 [1, 1, 21],
 [1, 1, 22],
 [1, 1, 23],
 [1, 1, 24],
 [1, 1, 25],
 [1, 1, 26],
 [1, 1, 27],
 [1, 1, 28],
 [1, 1, 29],
 [1, 1, 30],
 [1, 1, 31],
 [1, 1, 32],
 [1, 1, 33],
 [1, 1, 34],
 [1, 1, 35],
 [1, 1, 36],
 [1, 1, 37],
 [1, 1, 38],
 [1, 1, 39],
 [1, 1, 40],
 [1, 1, 41],
 [1, 1, 42],
 [1, 1, 43],
 [1, 1, 44],
 [1, 1, 45],
 [1, 1, 46],
 [1, 1, 47],
 [1, 1, 48],
 [1, 1, 49],
 [1, 1, 50],
 [1, 1, 51],
 [1, 1, 52],
 [1, 1, 53],
 [1, 1, 54],
 [1, 1, 55],
 [1, 1, 56],
 [1, 1, 57],
 [1, 1, 58],
 [1, 1, 59],
 [1, 1, 60],
 [1, 1, 61],
 [1, 1, 62],
 [1, 1, 63],
 [1, 1, 64],
 [1, 1, 65],
 [1, 1, 66],
 [1, 1, 67],
 [1, 1, 68],
 [1, 1, 69],
 [1, 1, 70],
 [1, 1, 71],
 [1, 1, 72],
 [1, 1, 73],
 [1, 1, 74],
 [1, 1, 75],
 [1, 1, 76],
 [1, 1, 77],
 [1, 1, 

In [58]:
%time
shape = (91,109,91) 
coordinate = []
for element in itertools.product(*(np.arange(1,x+1) for x in shape)):
    coordinate.append(list(element))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


In [59]:
coordinate

[[1, 1, 1],
 [1, 1, 2],
 [1, 1, 3],
 [1, 1, 4],
 [1, 1, 5],
 [1, 1, 6],
 [1, 1, 7],
 [1, 1, 8],
 [1, 1, 9],
 [1, 1, 10],
 [1, 1, 11],
 [1, 1, 12],
 [1, 1, 13],
 [1, 1, 14],
 [1, 1, 15],
 [1, 1, 16],
 [1, 1, 17],
 [1, 1, 18],
 [1, 1, 19],
 [1, 1, 20],
 [1, 1, 21],
 [1, 1, 22],
 [1, 1, 23],
 [1, 1, 24],
 [1, 1, 25],
 [1, 1, 26],
 [1, 1, 27],
 [1, 1, 28],
 [1, 1, 29],
 [1, 1, 30],
 [1, 1, 31],
 [1, 1, 32],
 [1, 1, 33],
 [1, 1, 34],
 [1, 1, 35],
 [1, 1, 36],
 [1, 1, 37],
 [1, 1, 38],
 [1, 1, 39],
 [1, 1, 40],
 [1, 1, 41],
 [1, 1, 42],
 [1, 1, 43],
 [1, 1, 44],
 [1, 1, 45],
 [1, 1, 46],
 [1, 1, 47],
 [1, 1, 48],
 [1, 1, 49],
 [1, 1, 50],
 [1, 1, 51],
 [1, 1, 52],
 [1, 1, 53],
 [1, 1, 54],
 [1, 1, 55],
 [1, 1, 56],
 [1, 1, 57],
 [1, 1, 58],
 [1, 1, 59],
 [1, 1, 60],
 [1, 1, 61],
 [1, 1, 62],
 [1, 1, 63],
 [1, 1, 64],
 [1, 1, 65],
 [1, 1, 66],
 [1, 1, 67],
 [1, 1, 68],
 [1, 1, 69],
 [1, 1, 70],
 [1, 1, 71],
 [1, 1, 72],
 [1, 1, 73],
 [1, 1, 74],
 [1, 1, 75],
 [1, 1, 76],
 [1, 1, 77],
 [1, 1, 

In [136]:
import itertools
def product(shape, axes):
    prod_trans = tuple(zip(*itertools.product(*(range(shape[axis]) for axis in axes))))
    
    print(prod_trans)
    
    prod_trans_ordered = [None] * len(axes)
    for i, axis in enumerate(axes):
        prod_trans_ordered[axis] = prod_trans[i]
    return zip(*prod_trans_ordered)

In [137]:
a = product(shape=(91,109,91), axes=(0,1,2))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [138]:
shape=(91,109,91)
axes=(2,1,0)
prod_trans = tuple(zip(*itertools.product(*(range(shape[axis]) for axis in axes))))
prod_trans_ordered = [None] * len(axes)
for i, axis in enumerate(axes):
    prod_trans_ordered[axis] = prod_trans[i]

In [139]:
prod_trans_ordered

[(0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,


In [156]:
import itertools
grid_width = 3
grid = []
for element in itertools.product(np.arange(1,grid_width+1), np.arange(1,grid_width+1), np.arange(1,grid_width+1)):
    grid.append(list(element))
grid_coord = np.array(grid)

In [157]:
grid_coord #(35937, 3)

array([[1, 1, 1],
       [1, 1, 2],
       [1, 1, 3],
       [1, 2, 1],
       [1, 2, 2],
       [1, 2, 3],
       [1, 3, 1],
       [1, 3, 2],
       [1, 3, 3],
       [2, 1, 1],
       [2, 1, 2],
       [2, 1, 3],
       [2, 2, 1],
       [2, 2, 2],
       [2, 2, 3],
       [2, 3, 1],
       [2, 3, 2],
       [2, 3, 3],
       [3, 1, 1],
       [3, 1, 2],
       [3, 1, 3],
       [3, 2, 1],
       [3, 2, 2],
       [3, 2, 3],
       [3, 3, 1],
       [3, 3, 2],
       [3, 3, 3]])

In [162]:
import itertools
grid_width = 3
grid = []
for element in itertools.product(np.arange(1,grid_width+1), np.arange(1,grid_width+1), np.arange(1,grid_width+1)):
    grid.append(list(element)[::-1])
grid_coord = np.array(grid)


In [163]:
grid_coord

array([[1, 1, 1],
       [2, 1, 1],
       [3, 1, 1],
       [1, 2, 1],
       [2, 2, 1],
       [3, 2, 1],
       [1, 3, 1],
       [2, 3, 1],
       [3, 3, 1],
       [1, 1, 2],
       [2, 1, 2],
       [3, 1, 2],
       [1, 2, 2],
       [2, 2, 2],
       [3, 2, 2],
       [1, 3, 2],
       [2, 3, 2],
       [3, 3, 2],
       [1, 1, 3],
       [2, 1, 3],
       [3, 1, 3],
       [1, 2, 3],
       [2, 2, 3],
       [3, 2, 3],
       [1, 3, 3],
       [2, 3, 3],
       [3, 3, 3]])

In [165]:
xx,yy,zz = 91,109,91
coordinate = []
for element in itertools.product(np.arange(1,xx+1), np.arange(1,yy+1), np.arange(1,zz+1)):
    coordinate.append(list(element)[::-1])

In [166]:
coordinate

[[1, 1, 1],
 [2, 1, 1],
 [3, 1, 1],
 [4, 1, 1],
 [5, 1, 1],
 [6, 1, 1],
 [7, 1, 1],
 [8, 1, 1],
 [9, 1, 1],
 [10, 1, 1],
 [11, 1, 1],
 [12, 1, 1],
 [13, 1, 1],
 [14, 1, 1],
 [15, 1, 1],
 [16, 1, 1],
 [17, 1, 1],
 [18, 1, 1],
 [19, 1, 1],
 [20, 1, 1],
 [21, 1, 1],
 [22, 1, 1],
 [23, 1, 1],
 [24, 1, 1],
 [25, 1, 1],
 [26, 1, 1],
 [27, 1, 1],
 [28, 1, 1],
 [29, 1, 1],
 [30, 1, 1],
 [31, 1, 1],
 [32, 1, 1],
 [33, 1, 1],
 [34, 1, 1],
 [35, 1, 1],
 [36, 1, 1],
 [37, 1, 1],
 [38, 1, 1],
 [39, 1, 1],
 [40, 1, 1],
 [41, 1, 1],
 [42, 1, 1],
 [43, 1, 1],
 [44, 1, 1],
 [45, 1, 1],
 [46, 1, 1],
 [47, 1, 1],
 [48, 1, 1],
 [49, 1, 1],
 [50, 1, 1],
 [51, 1, 1],
 [52, 1, 1],
 [53, 1, 1],
 [54, 1, 1],
 [55, 1, 1],
 [56, 1, 1],
 [57, 1, 1],
 [58, 1, 1],
 [59, 1, 1],
 [60, 1, 1],
 [61, 1, 1],
 [62, 1, 1],
 [63, 1, 1],
 [64, 1, 1],
 [65, 1, 1],
 [66, 1, 1],
 [67, 1, 1],
 [68, 1, 1],
 [69, 1, 1],
 [70, 1, 1],
 [71, 1, 1],
 [72, 1, 1],
 [73, 1, 1],
 [74, 1, 1],
 [75, 1, 1],
 [76, 1, 1],
 [77, 1, 1],
 [78, 1,

In [53]:
grid_coord.reshape((35937, 3),order='F')

array([[ 1,  1,  1],
       [ 1,  1,  2],
       [ 1,  1,  3],
       ...,
       [33, 33, 31],
       [33, 33, 32],
       [33, 33, 33]])

In [150]:
grid_width = 33
tmp = np.arange(start=1,stop=2*grid_width, step=2) #np array: [1, 3, 5, ..., 65]
tmp_coords = list()
for element in itertools.product(tmp,tmp,tmp):
    tmp_coords.append(list(element))
tmp_coords = np.array(tmp_coords)

In [151]:
tmp_coords

array([[ 1,  1,  1],
       [ 1,  1,  3],
       [ 1,  1,  5],
       ...,
       [65, 65, 61],
       [65, 65, 63],
       [65, 65, 65]])

In [140]:
import itertools
def product(shape, axes):
    prod_trans = tuple(zip(*itertools.product(*(range(shape[axis]) for axis in axes))))

    prod_trans_ordered = [None] * len(axes)
    for i, axis in enumerate(axes):
        prod_trans_ordered[axis] = prod_trans[i]
    return zip(*prod_trans_ordered)

In [141]:
axes = (2,1,0)
for i, axis in enumerate(axes):
    print(i, axis)
prod_trans_ordered = [None] * len(axes)
print(prod_trans_ordered)

0 2
1 1
2 0
[None, None, None]


In [149]:
a = zip(*itertools.product(*(range(shape[axis]) for axis in axes)))

In [147]:
%%time
product((33, 33, 33), (2,1,0))
print(*product((33, 33, 33), (2,1,0)))


(0, 0, 0) (1, 0, 0) (2, 0, 0) (3, 0, 0) (4, 0, 0) (5, 0, 0) (6, 0, 0) (7, 0, 0) (8, 0, 0) (9, 0, 0) (10, 0, 0) (11, 0, 0) (12, 0, 0) (13, 0, 0) (14, 0, 0) (15, 0, 0) (16, 0, 0) (17, 0, 0) (18, 0, 0) (19, 0, 0) (20, 0, 0) (21, 0, 0) (22, 0, 0) (23, 0, 0) (24, 0, 0) (25, 0, 0) (26, 0, 0) (27, 0, 0) (28, 0, 0) (29, 0, 0) (30, 0, 0) (31, 0, 0) (32, 0, 0) (0, 1, 0) (1, 1, 0) (2, 1, 0) (3, 1, 0) (4, 1, 0) (5, 1, 0) (6, 1, 0) (7, 1, 0) (8, 1, 0) (9, 1, 0) (10, 1, 0) (11, 1, 0) (12, 1, 0) (13, 1, 0) (14, 1, 0) (15, 1, 0) (16, 1, 0) (17, 1, 0) (18, 1, 0) (19, 1, 0) (20, 1, 0) (21, 1, 0) (22, 1, 0) (23, 1, 0) (24, 1, 0) (25, 1, 0) (26, 1, 0) (27, 1, 0) (28, 1, 0) (29, 1, 0) (30, 1, 0) (31, 1, 0) (32, 1, 0) (0, 2, 0) (1, 2, 0) (2, 2, 0) (3, 2, 0) (4, 2, 0) (5, 2, 0) (6, 2, 0) (7, 2, 0) (8, 2, 0) (9, 2, 0) (10, 2, 0) (11, 2, 0) (12, 2, 0) (13, 2, 0) (14, 2, 0) (15, 2, 0) (16, 2, 0) (17, 2, 0) (18, 2, 0) (19, 2, 0) (20, 2, 0) (21, 2, 0) (22, 2, 0) (23, 2, 0) (24, 2, 0) (25, 2, 0) (26, 2, 0) (27, 2,

(9, 31, 1) (10, 31, 1) (11, 31, 1) (12, 31, 1) (13, 31, 1) (14, 31, 1) (15, 31, 1) (16, 31, 1) (17, 31, 1) (18, 31, 1) (19, 31, 1) (20, 31, 1) (21, 31, 1) (22, 31, 1) (23, 31, 1) (24, 31, 1) (25, 31, 1) (26, 31, 1) (27, 31, 1) (28, 31, 1) (29, 31, 1) (30, 31, 1) (31, 31, 1) (32, 31, 1) (0, 32, 1) (1, 32, 1) (2, 32, 1) (3, 32, 1) (4, 32, 1) (5, 32, 1) (6, 32, 1) (7, 32, 1) (8, 32, 1) (9, 32, 1) (10, 32, 1) (11, 32, 1) (12, 32, 1) (13, 32, 1) (14, 32, 1) (15, 32, 1) (16, 32, 1) (17, 32, 1) (18, 32, 1) (19, 32, 1) (20, 32, 1) (21, 32, 1) (22, 32, 1) (23, 32, 1) (24, 32, 1) (25, 32, 1) (26, 32, 1) (27, 32, 1) (28, 32, 1) (29, 32, 1) (30, 32, 1) (31, 32, 1) (32, 32, 1) (0, 0, 2) (1, 0, 2) (2, 0, 2) (3, 0, 2) (4, 0, 2) (5, 0, 2) (6, 0, 2) (7, 0, 2) (8, 0, 2) (9, 0, 2) (10, 0, 2) (11, 0, 2) (12, 0, 2) (13, 0, 2) (14, 0, 2) (15, 0, 2) (16, 0, 2) (17, 0, 2) (18, 0, 2) (19, 0, 2) (20, 0, 2) (21, 0, 2) (22, 0, 2) (23, 0, 2) (24, 0, 2) (25, 0, 2) (26, 0, 2) (27, 0, 2) (28, 0, 2) (29, 0, 2) (30, 0,

 (24, 10, 3) (25, 10, 3) (26, 10, 3) (27, 10, 3) (28, 10, 3) (29, 10, 3) (30, 10, 3) (31, 10, 3) (32, 10, 3) (0, 11, 3) (1, 11, 3) (2, 11, 3) (3, 11, 3) (4, 11, 3) (5, 11, 3) (6, 11, 3) (7, 11, 3) (8, 11, 3) (9, 11, 3) (10, 11, 3) (11, 11, 3) (12, 11, 3) (13, 11, 3) (14, 11, 3) (15, 11, 3) (16, 11, 3) (17, 11, 3) (18, 11, 3) (19, 11, 3) (20, 11, 3) (21, 11, 3) (22, 11, 3) (23, 11, 3) (24, 11, 3) (25, 11, 3) (26, 11, 3) (27, 11, 3) (28, 11, 3) (29, 11, 3) (30, 11, 3) (31, 11, 3) (32, 11, 3) (0, 12, 3) (1, 12, 3) (2, 12, 3) (3, 12, 3) (4, 12, 3) (5, 12, 3) (6, 12, 3) (7, 12, 3) (8, 12, 3) (9, 12, 3) (10, 12, 3) (11, 12, 3) (12, 12, 3) (13, 12, 3) (14, 12, 3) (15, 12, 3) (16, 12, 3) (17, 12, 3) (18, 12, 3) (19, 12, 3) (20, 12, 3) (21, 12, 3) (22, 12, 3) (23, 12, 3) (24, 12, 3) (25, 12, 3) (26, 12, 3) (27, 12, 3) (28, 12, 3) (29, 12, 3) (30, 12, 3) (31, 12, 3) (32, 12, 3) (0, 13, 3) (1, 13, 3) (2, 13, 3) (3, 13, 3) (4, 13, 3) (5, 13, 3) (6, 13, 3) (7, 13, 3) (8, 13, 3) (9, 13, 3) (10, 13, 

 (7, 2, 5) (8, 2, 5) (9, 2, 5) (10, 2, 5) (11, 2, 5) (12, 2, 5) (13, 2, 5) (14, 2, 5) (15, 2, 5) (16, 2, 5) (17, 2, 5) (18, 2, 5) (19, 2, 5) (20, 2, 5) (21, 2, 5) (22, 2, 5) (23, 2, 5) (24, 2, 5) (25, 2, 5) (26, 2, 5) (27, 2, 5) (28, 2, 5) (29, 2, 5) (30, 2, 5) (31, 2, 5) (32, 2, 5) (0, 3, 5) (1, 3, 5) (2, 3, 5) (3, 3, 5) (4, 3, 5) (5, 3, 5) (6, 3, 5) (7, 3, 5) (8, 3, 5) (9, 3, 5) (10, 3, 5) (11, 3, 5) (12, 3, 5) (13, 3, 5) (14, 3, 5) (15, 3, 5) (16, 3, 5) (17, 3, 5) (18, 3, 5) (19, 3, 5) (20, 3, 5) (21, 3, 5) (22, 3, 5) (23, 3, 5) (24, 3, 5) (25, 3, 5) (26, 3, 5) (27, 3, 5) (28, 3, 5) (29, 3, 5) (30, 3, 5) (31, 3, 5) (32, 3, 5) (0, 4, 5) (1, 4, 5) (2, 4, 5) (3, 4, 5) (4, 4, 5) (5, 4, 5) (6, 4, 5) (7, 4, 5) (8, 4, 5) (9, 4, 5) (10, 4, 5) (11, 4, 5) (12, 4, 5) (13, 4, 5) (14, 4, 5) (15, 4, 5) (16, 4, 5) (17, 4, 5) (18, 4, 5) (19, 4, 5) (20, 4, 5) (21, 4, 5) (22, 4, 5) (23, 4, 5) (24, 4, 5) (25, 4, 5) (26, 4, 5) (27, 4, 5) (28, 4, 5) (29, 4, 5) (30, 4, 5) (31, 4, 5) (32, 4, 5) (0, 5, 5) 

 (25, 17, 6) (26, 17, 6) (27, 17, 6) (28, 17, 6) (29, 17, 6) (30, 17, 6) (31, 17, 6) (32, 17, 6) (0, 18, 6) (1, 18, 6) (2, 18, 6) (3, 18, 6) (4, 18, 6) (5, 18, 6) (6, 18, 6) (7, 18, 6) (8, 18, 6) (9, 18, 6) (10, 18, 6) (11, 18, 6) (12, 18, 6) (13, 18, 6) (14, 18, 6) (15, 18, 6) (16, 18, 6) (17, 18, 6) (18, 18, 6) (19, 18, 6) (20, 18, 6) (21, 18, 6) (22, 18, 6) (23, 18, 6) (24, 18, 6) (25, 18, 6) (26, 18, 6) (27, 18, 6) (28, 18, 6) (29, 18, 6) (30, 18, 6) (31, 18, 6) (32, 18, 6) (0, 19, 6) (1, 19, 6) (2, 19, 6) (3, 19, 6) (4, 19, 6) (5, 19, 6) (6, 19, 6) (7, 19, 6) (8, 19, 6) (9, 19, 6) (10, 19, 6) (11, 19, 6) (12, 19, 6) (13, 19, 6) (14, 19, 6) (15, 19, 6) (16, 19, 6) (17, 19, 6) (18, 19, 6) (19, 19, 6) (20, 19, 6) (21, 19, 6) (22, 19, 6) (23, 19, 6) (24, 19, 6) (25, 19, 6) (26, 19, 6) (27, 19, 6) (28, 19, 6) (29, 19, 6) (30, 19, 6) (31, 19, 6) (32, 19, 6) (0, 20, 6) (1, 20, 6) (2, 20, 6) (3, 20, 6) (4, 20, 6) (5, 20, 6) (6, 20, 6) (7, 20, 6) (8, 20, 6) (9, 20, 6) (10, 20, 6) (11, 20, 

...

(26, 11, 25) (27, 11, 25) (28, 11, 25) (29, 11, 25) (30, 11, 25) (31, 11, 25) (32, 11, 25) (0, 12, 25) (1, 12, 25) (2, 12, 25) (3, 12, 25) (4, 12, 25) (5, 12, 25) (6, 12, 25) (7, 12, 25) (8, 12, 25) (9, 12, 25) (10, 12, 25) (11, 12, 25) (12, 12, 25) (13, 12, 25) (14, 12, 25) (15, 12, 25) (16, 12, 25) (17, 12, 25) (18, 12, 25) (19, 12, 25) (20, 12, 25) (21, 12, 25) (22, 12, 25) (23, 12, 25) (24, 12, 25) (25, 12, 25) (26, 12, 25) (27, 12, 25) (28, 12, 25) (29, 12, 25) (30, 12, 25) (31, 12, 25) (32, 12, 25) (0, 13, 25) (1, 13, 25) (2, 13, 25) (3, 13, 25) (4, 13, 25) (5, 13, 25) (6, 13, 25) (7, 13, 25) (8, 13, 25) (9, 13, 25) (10, 13, 25) (11, 13, 25) (12, 13, 25) (13, 13, 25) (14, 13, 25) (15, 13, 25) (16, 13, 25) (17, 13, 25) (18, 13, 25) (19, 13, 25) (20, 13, 25) (21, 13, 25) (22, 13, 25) (23, 13, 25) (24, 13, 25) (25, 13, 25) (26, 13, 25) (27, 13, 25) (28, 13, 25) (29, 13, 25) (30, 13, 25) (31, 13, 25) (32, 13, 25) (0, 14, 25) (1, 14, 25) (2, 14, 25) (3, 14, 25) (4, 14, 25) (5, 14, 25)

 (13, 6, 27) (14, 6, 27) (15, 6, 27) (16, 6, 27) (17, 6, 27) (18, 6, 27) (19, 6, 27) (20, 6, 27) (21, 6, 27) (22, 6, 27) (23, 6, 27) (24, 6, 27) (25, 6, 27) (26, 6, 27) (27, 6, 27) (28, 6, 27) (29, 6, 27) (30, 6, 27) (31, 6, 27) (32, 6, 27) (0, 7, 27) (1, 7, 27) (2, 7, 27) (3, 7, 27) (4, 7, 27) (5, 7, 27) (6, 7, 27) (7, 7, 27) (8, 7, 27) (9, 7, 27) (10, 7, 27) (11, 7, 27) (12, 7, 27) (13, 7, 27) (14, 7, 27) (15, 7, 27) (16, 7, 27) (17, 7, 27) (18, 7, 27) (19, 7, 27) (20, 7, 27) (21, 7, 27) (22, 7, 27) (23, 7, 27) (24, 7, 27) (25, 7, 27) (26, 7, 27) (27, 7, 27) (28, 7, 27) (29, 7, 27) (30, 7, 27) (31, 7, 27) (32, 7, 27) (0, 8, 27) (1, 8, 27) (2, 8, 27) (3, 8, 27) (4, 8, 27) (5, 8, 27) (6, 8, 27) (7, 8, 27) (8, 8, 27) (9, 8, 27) (10, 8, 27) (11, 8, 27) (12, 8, 27) (13, 8, 27) (14, 8, 27) (15, 8, 27) (16, 8, 27) (17, 8, 27) (18, 8, 27) (19, 8, 27) (20, 8, 27) (21, 8, 27) (22, 8, 27) (23, 8, 27) (24, 8, 27) (25, 8, 27) (26, 8, 27) (27, 8, 27) (28, 8, 27) (29, 8, 27) (30, 8, 27) (31, 8, 27)

 (20, 32, 28) (21, 32, 28) (22, 32, 28) (23, 32, 28) (24, 32, 28) (25, 32, 28) (26, 32, 28) (27, 32, 28) (28, 32, 28) (29, 32, 28) (30, 32, 28) (31, 32, 28) (32, 32, 28) (0, 0, 29) (1, 0, 29) (2, 0, 29) (3, 0, 29) (4, 0, 29) (5, 0, 29) (6, 0, 29) (7, 0, 29) (8, 0, 29) (9, 0, 29) (10, 0, 29) (11, 0, 29) (12, 0, 29) (13, 0, 29) (14, 0, 29) (15, 0, 29) (16, 0, 29) (17, 0, 29) (18, 0, 29) (19, 0, 29) (20, 0, 29) (21, 0, 29) (22, 0, 29) (23, 0, 29) (24, 0, 29) (25, 0, 29) (26, 0, 29) (27, 0, 29) (28, 0, 29) (29, 0, 29) (30, 0, 29) (31, 0, 29) (32, 0, 29) (0, 1, 29) (1, 1, 29) (2, 1, 29) (3, 1, 29) (4, 1, 29) (5, 1, 29) (6, 1, 29) (7, 1, 29) (8, 1, 29) (9, 1, 29) (10, 1, 29) (11, 1, 29) (12, 1, 29) (13, 1, 29) (14, 1, 29) (15, 1, 29) (16, 1, 29) (17, 1, 29) (18, 1, 29) (19, 1, 29) (20, 1, 29) (21, 1, 29) (22, 1, 29) (23, 1, 29) (24, 1, 29) (25, 1, 29) (26, 1, 29) (27, 1, 29) (28, 1, 29) (29, 1, 29) (30, 1, 29) (31, 1, 29) (32, 1, 29) (0, 2, 29) (1, 2, 29) (2, 2, 29) (3, 2, 29) (4, 2, 29) (5,

 (14, 13, 30) (15, 13, 30) (16, 13, 30) (17, 13, 30) (18, 13, 30) (19, 13, 30) (20, 13, 30) (21, 13, 30) (22, 13, 30) (23, 13, 30) (24, 13, 30) (25, 13, 30) (26, 13, 30) (27, 13, 30) (28, 13, 30) (29, 13, 30) (30, 13, 30) (31, 13, 30) (32, 13, 30) (0, 14, 30) (1, 14, 30) (2, 14, 30) (3, 14, 30) (4, 14, 30) (5, 14, 30) (6, 14, 30) (7, 14, 30) (8, 14, 30) (9, 14, 30) (10, 14, 30) (11, 14, 30) (12, 14, 30) (13, 14, 30) (14, 14, 30) (15, 14, 30) (16, 14, 30) (17, 14, 30) (18, 14, 30) (19, 14, 30) (20, 14, 30) (21, 14, 30) (22, 14, 30) (23, 14, 30) (24, 14, 30) (25, 14, 30) (26, 14, 30) (27, 14, 30) (28, 14, 30) (29, 14, 30) (30, 14, 30) (31, 14, 30) (32, 14, 30) (0, 15, 30) (1, 15, 30) (2, 15, 30) (3, 15, 30) (4, 15, 30) (5, 15, 30) (6, 15, 30) (7, 15, 30) (8, 15, 30) (9, 15, 30) (10, 15, 30) (11, 15, 30) (12, 15, 30) (13, 15, 30) (14, 15, 30) (15, 15, 30) (16, 15, 30) (17, 15, 30) (18, 15, 30) (19, 15, 30) (20, 15, 30) (21, 15, 30) (22, 15, 30) (23, 15, 30) (24, 15, 30) (25, 15, 30) (26, 

(0, 8, 32) (1, 8, 32) (2, 8, 32) (3, 8, 32) (4, 8, 32) (5, 8, 32) (6, 8, 32) (7, 8, 32) (8, 8, 32) (9, 8, 32) (10, 8, 32) (11, 8, 32) (12, 8, 32) (13, 8, 32) (14, 8, 32) (15, 8, 32) (16, 8, 32) (17, 8, 32) (18, 8, 32) (19, 8, 32) (20, 8, 32) (21, 8, 32) (22, 8, 32) (23, 8, 32) (24, 8, 32) (25, 8, 32) (26, 8, 32) (27, 8, 32) (28, 8, 32) (29, 8, 32) (30, 8, 32) (31, 8, 32) (32, 8, 32) (0, 9, 32) (1, 9, 32) (2, 9, 32) (3, 9, 32) (4, 9, 32) (5, 9, 32) (6, 9, 32) (7, 9, 32) (8, 9, 32) (9, 9, 32) (10, 9, 32) (11, 9, 32) (12, 9, 32) (13, 9, 32) (14, 9, 32) (15, 9, 32) (16, 9, 32) (17, 9, 32) (18, 9, 32) (19, 9, 32) (20, 9, 32) (21, 9, 32) (22, 9, 32) (23, 9, 32) (24, 9, 32) (25, 9, 32) (26, 9, 32) (27, 9, 32) (28, 9, 32) (29, 9, 32) (30, 9, 32) (31, 9, 32) (32, 9, 32) (0, 10, 32) (1, 10, 32) (2, 10, 32) (3, 10, 32) (4, 10, 32) (5, 10, 32) (6, 10, 32) (7, 10, 32) (8, 10, 32) (9, 10, 32) (10, 10, 32) (11, 10, 32) (12, 10, 32) (13, 10, 32) (14, 10, 32) (15, 10, 32) (16, 10, 32) (17, 10, 32) (18,

In [23]:
n_total = 40
n_valid = 22
sigma = 10
grid_width=33

foci_generator = Foci_generator(population_center, probs, mask_file, sigma, grid_width)
foci = foci_generator.sample_drawer(n_total, n_valid)
print(foci)
foci = foci_dataset(foci)
areas = foci_generator.areas(radius=17)

ale = nimare.meta.cbma.ALE()
result = ale.fit(foci)

[[51. 68. 68.  0.]
 [30. 73. 37.  0.]
 [53. 70. 25.  0.]
 ...
 [34. 30. 41. 38.]
 [30. 35. 33. 39.]
 [28. 88. 37. 39.]]


In [24]:
foci.coordinates

,id,study_id,contrast_id,x,y,z,space,i,j,k
0,study0-0,study0,0,51.0,68.0,68.0,mni152_2mm,19,97,70
1,study0-0,study0,0,30.0,73.0,37.0,mni152_2mm,30,99,54
2,study0-0,study0,0,53.0,70.0,25.0,mni152_2mm,18,98,48
3,study1-6,study1,6,54.0,36.0,57.0,mni152_2mm,18,81,64
4,study1-6,study1,6,30.0,73.0,28.0,mni152_2mm,30,99,50
...,...,...,...,...,...,...,...,...,...,...
255,study38-1,study38,1,35.0,66.0,33.0,mni152_2mm,27,96,52
256,study38-1,study38,1,8.0,38.0,57.0,mni152_2mm,41,82,64
257,study38-1,study38,1,34.0,30.0,41.0,mni152_2mm,28,78,56
258,study39-1,study39,1,30.0,35.0,33.0,mni152_2mm,30,80,52


In [25]:
result.maps

{'ale': array([0., 0., 0., ..., 0., 0., 0.]),
 'p': array([1., 1., 1., ..., 1., 1., 1.]),
 'z': array([0., 0., 0., ..., 0., 0., 0.])}

In [26]:
result_array = result.get_map(name='p', return_type='array')
print(np.max(result_array))
print(np.min(result_array))
print(result_array.shape)
print(np.where(result_array ==1))

1.0
3.4930855264057177e-38
(352328,)
(array([     0,      1,      2, ..., 352325, 352326, 352327]),)


In [28]:
import nibabel as nib

mask = nib.load(mask_file) #type: nibabel.nifti1.Nifti1Image
mask_array = np.array(mask.dataobj)
#mask_array[mask_array>0] = 1

In [29]:
population_center

array([[63, 67, 50],
       [63, 35, 50],
       [31, 73, 31],
       [58, 74, 31],
       [28, 71, 50],
       [28, 38, 50],
       [57, 44, 66],
       [34, 44, 66]])

In [30]:
mask_array[31, 73, 31]

1.0000000591389835

In [31]:
mask_array[mask_array>0] = 1

In [32]:
mask_array[50][50]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [33]:
q = np.where(mask_array==1)

In [64]:
q[0].shape

(352328,)

In [74]:
i = 1
print(q[0][i], q[1][i], q[2][i])

4 43 34


In [67]:
i = 186250
print(q[0][i], q[1][i], q[2][i])

46 83 64


In [53]:
result_array[3467]

1.0

In [36]:
p_mask[12, 48, 53]

1.0

In [89]:
n_total = 40
n_valid = 22
sigma = 10
grid_width=33

foci_generator = Foci_generator(population_center, probs, mask_file, sigma, grid_width)
foci = foci_generator.sample_drawer(n_total, n_valid)
foci = foci_dataset(foci)

areas = foci_generator.areas(radius=17)

ale = nimare.meta.cbma.ALE(sample_size=25)
result = ale.fit(foci)
result_array = result.get_map(name='p', return_type='array')
"""
a = (result_array > 0.05).sum()
b = ((result_array <= 0.05) & (result_array>0)).sum() #reject
print(a,b)
print(np.where((result_array<=0.05) & (result_array>0)))
"""

'\na = (result_array > 0.05).sum()\nb = ((result_array <= 0.05) & (result_array>0)).sum() #reject\nprint(a,b)\nprint(np.where((result_array<=0.05) & (result_array>0)))\n'

In [87]:
p_mask = mask_array.copy()

q = np.where(p_mask==1)
print(q)
n_brain_voxels = q[1].shape[0]

q_xx, q_yy, q_zz = q[0], q[1], q[2]


(array([ 4,  4,  4, ..., 84, 84, 84]), array([43, 43, 43, ..., 55, 55, 55]), array([33, 34, 35, ..., 34, 35, 36]))


In [88]:
n_brain_voxels

304656

In [72]:
# replace the nonzero voxels in brain mask with its corrected p value
p_mask = mask_array.copy()
np.place(p_mask, p_mask==1, result_array)

In [73]:
p_mask[46,83,64]

0.09981560285513673

In [177]:
from nimare.correct import FWECorrector

corrector = FWECorrector(method='montecarlo',n_iters=100, n_cores=-1)
corrected_result = corrector.transform(result)
print(corrected_result.maps)
corrected_p_value = corrected_result.get_map(name='p', return_type='array') #shape:(352328,) with the given brain mask

INFO:nimare.correct:Using correction method implemented in Estimator: <class 'nimare.meta.cbma.ale.ALE'>.correct_fwe_montecarlo.


/opt/anaconda3/lib/python3.7/site-packages/nimare/meta/cbma/ale.py:360: RuntimeWarning: divide by zero encountered in log
  logp_cfwe_values = -np.log(p_cfwe_values)


{'ale': array([0., 0., 0., ..., 0., 0., 0.]), 'p': array([1., 1., 1., ..., 1., 1., 1.]), 'z': array([0., 0., 0., ..., 0., 0., 0.]), 'logp_level-voxel_corr-FWE_method-montecarlo': array([-0., -0., -0., ..., -0., -0., -0.]), 'z_level-voxel_corr-FWE_method-montecarlo': array([0., 0., 0., ..., 0., 0., 0.]), 'logp_level-cluster_corr-FWE_method-montecarlo': array([-0., -0., -0., ..., -0., -0., -0.]), 'z_level-cluster_corr-FWE_method-montecarlo': array([0., 0., 0., ..., 0., 0., 0.])}


/opt/anaconda3/lib/python3.7/site-packages/nimare/meta/cbma/ale.py:372: RuntimeWarning: divide by zero encountered in log
  logp_vfwe_values = -np.log(p_vfwe_values)


In [178]:
corrected_p_value.shape

(352328,)

In [179]:
# replace the nonzero voxels in brain mask with its corrected p value
corrected_p_mask = mask_array
np.place(corrected_p_mask, corrected_p_mask==1, corrected_p_value)

In [181]:
corrected_p_mask[30,82,61]

4.37636289996449e-36

In [135]:
corrected_p_mask[68,70,54]

1.0

In [134]:
center_p = list()
for j in range(8):
    center = population_center[j]
    p = corrected_p_mask[center[0],center[1],center[2]]
    center_p.append(p)
    print((corrected_p_mask[areas[j]>0] <= 0.05).sum())
print(center_p)

0
0
0
0
0
0
0
0
[1.0, 1.0, 1.0, 1.0, 0.06882726233733874, 1.0, 1.0, 1.0]


In [18]:
# replace the nonzero voxels in brain mask with its corrected p value
corrected_p_mask = mask_array
np.place(corrected_p_mask, corrected_p_mask==1, corrected_p_value)

In [19]:
np.where(corrected_p_mask==2)

(array([63]), array([67]), array([50]))

In [ ]:
# replace the nonzero voxels in brain mask with its corrected p value
corrected_p_mask = mask_array
np.place(corrected_p_mask, corrected_p_mask==1, corrected_p_value)


In [122]:
n_total = 40
n_valid = 22
sigma = 10
grid_width=33

foci_generator = Foci_generator(population_center, probs, mask_file, sigma, grid_width)
foci = foci_generator.sample_drawer(n_total, n_valid)
print(foci)

[[63. 66. 52.  0.]
 [66. 66. 59.  0.]
 [63. 38. 51.  0.]
 ...
 [63. 51. 42. 39.]
 [61. 32. 58. 39.]
 [31. 60. 44. 39.]]


In [123]:
foci = foci_dataset(foci)


areas = foci_generator.areas(radius=17)

ale = nimare.meta.cbma.ALE(sample_size=25)
result = ale.fit(foci)
result_array = result.get_map(name='p', return_type='array')

In [124]:
coordinates = foci.coordinates

In [125]:
coordinates

,id,study_id,contrast_id,x,y,z,space,i,j,k
0,study0-0,study0,0,63.0,66.0,52.0,mni152_2mm,13,96,62
1,study0-0,study0,0,66.0,66.0,59.0,mni152_2mm,12,96,65
2,study0-0,study0,0,63.0,38.0,51.0,mni152_2mm,13,82,61
3,study0-0,study0,0,39.0,79.0,29.0,mni152_2mm,25,102,50
4,study0-0,study0,0,35.0,66.0,48.0,mni152_2mm,27,96,60
...,...,...,...,...,...,...,...,...,...,...
261,study39-0,study39,0,76.0,50.0,51.0,mni152_2mm,7,88,61
262,study39-0,study39,0,15.0,29.0,22.0,mni152_2mm,37,77,47
263,study39-0,study39,0,63.0,51.0,42.0,mni152_2mm,13,88,57
264,study39-0,study39,0,61.0,32.0,58.0,mni152_2mm,14,79,65


In [126]:
for id_, data in coordinates.groupby('id'):
    #print(id_, data)
    ijk = np.vstack((data.i.values, data.j.values, data.k.values)).T.astype(int)
    print(ijk)
    print('----------------')

[[ 13  96  62]
 [ 12  96  65]
 [ 13  82  61]
 [ 25 102  50]
 [ 27  96  60]
 [ 31  83  61]
 [ 16  87  66]]
----------------
[[ 12  97  59]
 [ 12  81  61]
 [ 14 103  49]]
----------------
[[ 12  98  60]
 [ 30 100  61]
 [ 31  83  61]
 [ 26  84  69]]
----------------
[[ 13  81  59]
 [ 28  98  52]
 [ 16 100  52]
 [ 28  99  62]
 [ 15  89  69]
 [ 27  87  68]]
----------------
[[ 12  95  60]
 [ 14  80  61]
 [ 29 101  51]
 [ 15 103  52]
 [ 15  99  52]
 [ 13 104  53]
 [ 17  87  70]]
----------------
[[ 14 100  60]
 [ 28  97  52]
 [ 15 100  52]
 [ 15  99  52]
 [ 32  99  64]
 [ 33  82  60]
 [ 28  81  61]]
----------------
[[ 14  95  63]
 [ 14  80  59]
 [ 18 100  53]
 [ 30  99  59]
 [ 16  86  69]
 [ 25  87  69]]
----------------
[[ 29 102  52]
 [ 30  99  53]
 [ 17 101  52]
 [ 30 100  59]
 [ 16  85  66]]
----------------
[[ 10  98  59]
 [ 13  80  60]
 [ 16  85  61]
 [ 11  82  58]
 [ 28 100  50]
 [ 28  98  65]
 [ 29  80  60]
 [ 27  86  72]]
----------------
[[ 13  99  63]
 [ 30 100  52]
 [ 18  99  52

In [117]:
 for j_peak in range(ijk.shape[0]):
        i, j, k = ijk[j_peak, :]
        print(i, j, k)

16 95 60
28 99 51
14 100 51
31 81 61
32 84 63
16 84 70
30 88 69
30 83 68
26 84 69


In [128]:
mask = foci.masker.mask_img
mask_data = mask.get_fdata().astype(float)

In [129]:
mask_data.shape

(91, 109, 91)

In [130]:
result_array.shape

(352328,)

In [131]:
(result_array<0.05).sum()

12899

In [132]:
np.min(result_array)

5.999141534077163e-29

In [133]:
import nibabel as nib
mask = nib.load(mask_file) #type: nibabel.nifti1.Nifti1Image
mask_array = np.array(mask.dataobj)
mask_array[mask_array>0] = 1 #only keep the value 0/1

In [134]:
mask_data = mask.get_fdata().astype(np.bool)

In [100]:
(mask_data == True).sum()

352328

In [55]:
mask_array.shape

(91, 109, 91)

In [115]:
mask_array[13, 96, 63]

0.0

In [59]:
# coordinates for brain voxels
p_mask = mask_array.copy()

q = np.where(p_mask==1)
print(q)
n_brain_voxels = q[0].shape[0] #352328

q_xx, q_yy, q_zz = q[0], q[1], q[2]

(array([ 4,  4,  4, ..., 84, 84, 84]), array([43, 43, 43, ..., 55, 55, 55]), array([33, 34, 35, ..., 34, 35, 36]))


In [85]:
#index of [63,67,50] is 285067
print(q_xx[285067], q_yy[285067], q_zz[285067])

63 67 50


In [86]:
result_array[285067]

1.0

In [157]:
corrected_p_mask = mask_array.copy()
print((corrected_p_mask).sum())
np.place(corrected_p_mask, corrected_p_mask==1, result_array)

352328.0


In [146]:
corrected_p_mask[37,77,47]

0.00012547310495400357

In [147]:
mask_array[33,74,40]

1.0

In [12]:
corrected_p_mask = mask_array.copy()
for i in range(n_brain_voxels):
    
    corrected_p_mask[q_zz[i], q_yy[i], q_xx[i]] = result_array[i]

In [133]:
u = np.where((corrected_p_mask>0) & (corrected_p_mask < 0.05))

In [135]:
u[0].shape

(13357,)

In [136]:
for j in range(13357):
    print([u[0][j], u[1][j], u[2][j]])

[36, 83, 27]
[36, 83, 28]
[36, 83, 29]
[36, 84, 26]
[36, 84, 27]
[36, 84, 28]
[36, 84, 29]
[36, 84, 30]
[36, 85, 26]
[36, 85, 27]
[36, 85, 28]
[36, 85, 29]
[36, 85, 30]
[36, 86, 26]
[36, 86, 27]
[36, 86, 28]
[36, 86, 29]
[36, 86, 30]
[36, 87, 27]
[36, 87, 28]
[36, 87, 29]
[37, 82, 27]
[37, 82, 28]
[37, 82, 29]
[37, 83, 26]
[37, 83, 27]
[37, 83, 28]
[37, 83, 29]
[37, 83, 30]
[37, 84, 25]
[37, 84, 26]
[37, 84, 27]
[37, 84, 28]
[37, 84, 29]
[37, 84, 30]
[37, 84, 31]
[37, 85, 25]
[37, 85, 26]
[37, 85, 27]
[37, 85, 28]
[37, 85, 29]
[37, 85, 30]
[37, 85, 31]
[37, 86, 25]
[37, 86, 26]
[37, 86, 27]
[37, 86, 28]
[37, 86, 29]
[37, 86, 30]
[37, 86, 31]
[37, 87, 26]
[37, 87, 27]
[37, 87, 28]
[37, 87, 29]
[37, 87, 30]
[37, 88, 27]
[37, 88, 28]
[37, 88, 29]
[38, 82, 26]
[38, 82, 27]
[38, 82, 28]
[38, 82, 29]
[38, 82, 30]
[38, 83, 25]
[38, 83, 26]
[38, 83, 27]
[38, 83, 28]
[38, 83, 29]
[38, 83, 30]
[38, 83, 31]
[38, 84, 25]
[38, 84, 26]
[38, 84, 27]
[38, 84, 28]
[38, 84, 29]
[38, 84, 30]
[38, 84, 31]

[45, 83, 32]
[45, 83, 33]
[45, 84, 13]
[45, 84, 14]
[45, 84, 15]
[45, 84, 16]
[45, 84, 17]
[45, 84, 18]
[45, 84, 19]
[45, 84, 20]
[45, 84, 21]
[45, 84, 22]
[45, 84, 23]
[45, 84, 24]
[45, 84, 25]
[45, 84, 26]
[45, 85, 13]
[45, 85, 14]
[45, 85, 15]
[45, 85, 16]
[45, 85, 17]
[45, 85, 18]
[45, 85, 19]
[45, 85, 20]
[45, 85, 21]
[45, 85, 22]
[45, 85, 23]
[45, 85, 24]
[45, 85, 25]
[45, 86, 14]
[45, 86, 15]
[45, 86, 16]
[45, 86, 17]
[45, 86, 18]
[45, 86, 19]
[45, 86, 20]
[45, 86, 21]
[45, 86, 22]
[45, 86, 23]
[45, 86, 24]
[45, 87, 15]
[45, 87, 16]
[45, 87, 17]
[45, 87, 18]
[45, 87, 19]
[45, 87, 20]
[45, 87, 21]
[45, 87, 22]
[45, 87, 23]
[45, 87, 37]
[45, 87, 38]
[45, 87, 39]
[45, 87, 40]
[45, 87, 41]
[45, 88, 15]
[45, 88, 16]
[45, 88, 17]
[45, 88, 18]
[45, 88, 19]
[45, 88, 20]
[45, 88, 21]
[45, 88, 22]
[45, 88, 35]
[45, 88, 36]
[45, 88, 37]
[45, 88, 38]
[45, 88, 39]
[45, 88, 40]
[45, 88, 41]
[45, 88, 42]
[45, 89, 16]
[45, 89, 17]
[45, 89, 18]
[45, 89, 19]
[45, 89, 20]
[45, 89, 21]
[45, 89, 22]

[48, 91, 38]
[48, 91, 39]
[48, 91, 40]
[48, 92, 19]
[48, 92, 20]
[48, 92, 21]
[48, 92, 22]
[48, 92, 27]
[48, 92, 28]
[48, 92, 29]
[48, 92, 30]
[48, 92, 31]
[48, 92, 32]
[48, 92, 33]
[48, 92, 34]
[48, 92, 35]
[48, 92, 36]
[48, 92, 37]
[48, 92, 38]
[48, 92, 39]
[48, 92, 40]
[48, 93, 20]
[48, 93, 21]
[48, 93, 22]
[48, 93, 23]
[48, 93, 24]
[48, 93, 25]
[48, 93, 26]
[48, 93, 27]
[48, 93, 28]
[48, 93, 29]
[48, 93, 30]
[48, 93, 31]
[48, 93, 32]
[48, 93, 33]
[48, 93, 34]
[48, 93, 35]
[48, 93, 36]
[48, 93, 37]
[48, 93, 38]
[48, 93, 39]
[48, 93, 40]
[48, 94, 20]
[48, 94, 21]
[48, 94, 22]
[48, 94, 23]
[48, 94, 24]
[48, 94, 25]
[48, 94, 26]
[48, 94, 27]
[48, 94, 28]
[48, 94, 29]
[48, 94, 30]
[48, 94, 31]
[48, 94, 32]
[48, 94, 33]
[48, 94, 34]
[48, 94, 35]
[48, 94, 36]
[48, 94, 37]
[48, 94, 38]
[48, 94, 39]
[48, 94, 40]
[48, 95, 21]
[48, 95, 22]
[48, 95, 23]
[48, 95, 24]
[48, 95, 25]
[48, 95, 26]
[48, 95, 27]
[48, 95, 28]
[48, 95, 29]
[48, 95, 30]
[48, 95, 31]
[48, 95, 32]
[48, 95, 33]
[48, 95, 34]

[51, 84, 27]
[51, 84, 28]
[51, 84, 29]
[51, 84, 30]
[51, 85, 15]
[51, 85, 16]
[51, 85, 17]
[51, 85, 18]
[51, 85, 19]
[51, 85, 20]
[51, 85, 21]
[51, 85, 22]
[51, 85, 23]
[51, 85, 24]
[51, 85, 25]
[51, 85, 26]
[51, 85, 27]
[51, 85, 28]
[51, 85, 29]
[51, 86, 16]
[51, 86, 17]
[51, 86, 18]
[51, 86, 19]
[51, 86, 20]
[51, 86, 21]
[51, 86, 22]
[51, 86, 23]
[51, 86, 24]
[51, 86, 25]
[51, 86, 26]
[51, 86, 27]
[51, 86, 28]
[51, 87, 17]
[51, 87, 18]
[51, 87, 19]
[51, 87, 20]
[51, 87, 21]
[51, 87, 22]
[51, 87, 23]
[51, 87, 24]
[51, 87, 25]
[51, 87, 26]
[51, 87, 42]
[51, 87, 43]
[51, 87, 44]
[51, 88, 17]
[51, 88, 18]
[51, 88, 19]
[51, 88, 20]
[51, 88, 21]
[51, 88, 22]
[51, 88, 23]
[51, 88, 24]
[51, 88, 25]
[51, 88, 32]
[51, 88, 33]
[51, 88, 34]
[51, 88, 41]
[51, 88, 42]
[51, 88, 43]
[51, 88, 44]
[51, 88, 45]
[51, 89, 18]
[51, 89, 19]
[51, 89, 20]
[51, 89, 21]
[51, 89, 22]
[51, 89, 23]
[51, 89, 24]
[51, 89, 30]
[51, 89, 31]
[51, 89, 32]
[51, 89, 33]
[51, 89, 34]
[51, 89, 35]
[51, 89, 36]
[51, 89, 41]

[54, 95, 30]
[54, 95, 31]
[54, 95, 32]
[54, 95, 33]
[54, 95, 34]
[54, 95, 35]
[54, 95, 36]
[54, 95, 37]
[54, 95, 38]
[54, 95, 39]
[54, 95, 40]
[54, 95, 41]
[54, 96, 28]
[54, 96, 29]
[54, 96, 30]
[54, 96, 31]
[54, 96, 32]
[54, 96, 33]
[54, 96, 34]
[54, 96, 35]
[54, 96, 36]
[54, 96, 37]
[54, 96, 38]
[54, 96, 39]
[54, 96, 40]
[54, 96, 41]
[54, 96, 42]
[54, 97, 29]
[54, 97, 30]
[54, 97, 31]
[54, 97, 32]
[54, 97, 33]
[54, 97, 34]
[54, 97, 35]
[54, 97, 36]
[54, 97, 37]
[54, 97, 38]
[54, 97, 39]
[54, 97, 40]
[54, 97, 41]
[54, 97, 42]
[54, 97, 43]
[54, 98, 31]
[54, 98, 32]
[54, 98, 33]
[54, 98, 34]
[54, 98, 35]
[54, 98, 36]
[54, 98, 37]
[54, 98, 38]
[54, 98, 39]
[54, 98, 40]
[54, 98, 41]
[54, 98, 42]
[54, 98, 43]
[54, 99, 34]
[54, 99, 35]
[54, 99, 36]
[54, 99, 37]
[54, 99, 38]
[54, 99, 39]
[54, 99, 40]
[54, 99, 41]
[54, 99, 42]
[54, 99, 43]
[54, 100, 37]
[54, 100, 38]
[54, 100, 39]
[54, 100, 40]
[54, 100, 41]
[54, 100, 42]
[55, 66, 31]
[55, 66, 32]
[55, 66, 33]
[55, 66, 34]
[55, 66, 35]
[55, 6

[57, 84, 22]
[57, 84, 23]
[57, 84, 24]
[57, 84, 25]
[57, 84, 26]
[57, 84, 30]
[57, 84, 31]
[57, 84, 32]
[57, 84, 33]
[57, 84, 34]
[57, 84, 35]
[57, 84, 36]
[57, 84, 37]
[57, 84, 38]
[57, 84, 39]
[57, 84, 40]
[57, 84, 41]
[57, 85, 19]
[57, 85, 20]
[57, 85, 21]
[57, 85, 22]
[57, 85, 23]
[57, 85, 24]
[57, 85, 25]
[57, 85, 26]
[57, 85, 30]
[57, 85, 31]
[57, 85, 32]
[57, 85, 33]
[57, 85, 34]
[57, 85, 35]
[57, 85, 36]
[57, 85, 37]
[57, 85, 38]
[57, 85, 39]
[57, 85, 40]
[57, 85, 41]
[57, 86, 20]
[57, 86, 21]
[57, 86, 22]
[57, 86, 23]
[57, 86, 24]
[57, 86, 25]
[57, 86, 26]
[57, 86, 31]
[57, 86, 32]
[57, 86, 33]
[57, 86, 34]
[57, 86, 35]
[57, 86, 36]
[57, 86, 37]
[57, 86, 38]
[57, 86, 39]
[57, 86, 40]
[57, 86, 41]
[57, 86, 42]
[57, 87, 21]
[57, 87, 22]
[57, 87, 23]
[57, 87, 24]
[57, 87, 25]
[57, 87, 26]
[57, 87, 32]
[57, 87, 33]
[57, 87, 34]
[57, 87, 35]
[57, 87, 36]
[57, 87, 37]
[57, 87, 38]
[57, 87, 39]
[57, 87, 40]
[57, 87, 41]
[57, 87, 42]
[57, 87, 43]
[57, 87, 44]
[57, 88, 21]
[57, 88, 22]

[61, 87, 35]
[61, 87, 36]
[61, 87, 37]
[61, 87, 38]
[61, 87, 39]
[61, 87, 40]
[61, 87, 41]
[61, 88, 35]
[61, 88, 36]
[61, 88, 37]
[61, 88, 38]
[61, 88, 39]
[61, 88, 40]
[61, 88, 41]
[61, 89, 35]
[61, 89, 36]
[61, 89, 37]
[61, 89, 38]
[61, 89, 39]
[61, 89, 40]
[61, 89, 41]
[61, 90, 35]
[61, 90, 36]
[61, 90, 37]
[61, 90, 38]
[61, 90, 39]
[61, 90, 40]
[61, 90, 41]
[61, 91, 35]
[61, 91, 36]
[61, 91, 37]
[61, 91, 38]
[61, 91, 39]
[61, 91, 40]
[61, 91, 41]
[61, 92, 35]
[61, 92, 36]
[61, 92, 37]
[61, 92, 38]
[61, 92, 39]
[61, 92, 40]
[61, 92, 41]
[61, 93, 35]
[61, 93, 36]
[61, 93, 37]
[61, 93, 38]
[61, 93, 39]
[61, 93, 40]
[61, 93, 41]
[61, 94, 36]
[61, 94, 37]
[61, 94, 38]
[61, 94, 39]
[61, 94, 40]
[61, 94, 41]
[61, 95, 37]
[61, 95, 38]
[61, 95, 39]
[61, 95, 40]
[62, 64, 27]
[62, 64, 28]
[62, 64, 29]
[62, 65, 26]
[62, 65, 27]
[62, 65, 28]
[62, 65, 29]
[62, 65, 30]
[62, 66, 25]
[62, 66, 26]
[62, 66, 27]
[62, 66, 28]
[62, 66, 29]
[62, 66, 30]
[62, 66, 31]
[62, 66, 39]
[62, 67, 16]
[62, 67, 17]

[68, 74, 41]
[68, 74, 42]
[68, 74, 43]
[68, 74, 44]
[68, 75, 22]
[68, 75, 23]
[68, 75, 24]
[68, 75, 25]
[68, 75, 26]
[68, 75, 27]
[68, 75, 28]
[68, 75, 29]
[68, 75, 30]
[68, 75, 31]
[68, 75, 32]
[68, 75, 33]
[68, 75, 34]
[68, 75, 39]
[68, 75, 40]
[68, 75, 41]
[68, 75, 42]
[68, 75, 43]
[68, 75, 44]
[68, 75, 45]
[68, 76, 22]
[68, 76, 23]
[68, 76, 24]
[68, 76, 25]
[68, 76, 26]
[68, 76, 27]
[68, 76, 28]
[68, 76, 29]
[68, 76, 30]
[68, 76, 31]
[68, 76, 32]
[68, 76, 33]
[68, 76, 34]
[68, 76, 39]
[68, 76, 40]
[68, 76, 41]
[68, 76, 42]
[68, 76, 43]
[68, 76, 44]
[68, 76, 45]
[68, 77, 23]
[68, 77, 24]
[68, 77, 25]
[68, 77, 26]
[68, 77, 27]
[68, 77, 28]
[68, 77, 29]
[68, 77, 30]
[68, 77, 31]
[68, 77, 32]
[68, 77, 33]
[68, 77, 34]
[68, 77, 35]
[68, 77, 38]
[68, 77, 39]
[68, 77, 40]
[68, 77, 41]
[68, 77, 42]
[68, 77, 43]
[68, 77, 44]
[68, 77, 45]
[68, 78, 24]
[68, 78, 25]
[68, 78, 26]
[68, 78, 27]
[68, 78, 28]
[68, 78, 29]
[68, 78, 30]
[68, 78, 31]
[68, 78, 32]
[68, 78, 33]
[68, 78, 34]
[68, 78, 35]

In [137]:
corrected_p_mask[63,47,50]

1.0

In [150]:
import nibabel as nib
mask = nib.load(mask_file) #type: nibabel.nifti1.Nifti1Image
mask_array = np.array(mask.dataobj)
mask_array[mask_array>0] = 1 #only keep the value 0/1

In [152]:
affine = mask.affine

In [148]:
def mm2vox(xyz, affine):
    """
    Convert coordinates to matrix subscripts.
    From here:
    http://blog.chrisgorgolewski.org/2014/12/how-to-convert-between-voxel-and-mm.html
    """
    ijk = nib.affines.apply_affine(np.linalg.inv(affine), xyz).astype(int)
    return ijk

In [156]:
mm2vox(xyz=[63,67,50], affine=affine)

array([13, 96, 61])

In [158]:
corrected_p_mask[13, 96, 61]

0.0

In [159]:
a = [0.0, 0.0, 8.994939295810967e-14, 0.0, 0.0, 6.794507748477554e-16, 0.0, 0.0]

In [161]:
1[a[0] < 0.05]

TypeError: 'int' object is not subscriptable

In [162]:
corrected_p_mask.shape

(91, 109, 91)

In [163]:
corrected_p_mask

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [167]:
import itertools
grid_width = 33
tmp = np.arange(start=1,stop=grid_width+1, step=1) #np array: [1, 3, 5, ..., 65]
tmp_coords = list()
for element in itertools.product(tmp,tmp,tmp):
    tmp_coords.append(list(element))
tmp_coords = np.array(tmp_coords) #shape:(35937, 3)

In [168]:
print(tmp)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]


In [171]:
tmp_coords

array([[ 1,  1,  1],
       [ 1,  1,  2],
       [ 1,  1,  3],
       ...,
       [33, 33, 31],
       [33, 33, 32],
       [33, 33, 33]])

In [172]:
length = tmp_coords.shape[0]
center_coord = np.array([[17,17,17]]*length)

In [173]:
center_coord

array([[17, 17, 17],
       [17, 17, 17],
       [17, 17, 17],
       ...,
       [17, 17, 17],
       [17, 17, 17],
       [17, 17, 17]])

In [179]:
dist = np.linalg.norm(tmp_coords - center_coord, axis=1)

In [182]:
dist.reshape((33,33,33))

array([[[27.71281292, 27.14774392, 26.60826939, ..., 26.60826939,
         27.14774392, 27.71281292],
        [27.14774392, 26.57066051, 26.01922366, ..., 26.01922366,
         26.57066051, 27.14774392],
        [26.60826939, 26.01922366, 25.45584412, ..., 25.45584412,
         26.01922366, 26.60826939],
        ...,
        [26.60826939, 26.01922366, 25.45584412, ..., 25.45584412,
         26.01922366, 26.60826939],
        [27.14774392, 26.57066051, 26.01922366, ..., 26.01922366,
         26.57066051, 27.14774392],
        [27.71281292, 27.14774392, 26.60826939, ..., 26.60826939,
         27.14774392, 27.71281292]],

       [[27.14774392, 26.57066051, 26.01922366, ..., 26.01922366,
         26.57066051, 27.14774392],
        [26.57066051, 25.98076211, 25.41653005, ..., 25.41653005,
         25.98076211, 26.57066051],
        [26.01922366, 25.41653005, 24.8394847 , ..., 24.8394847 ,
         25.41653005, 26.01922366],
        ...,
        [26.01922366, 25.41653005, 24.8394847 , ..., 2

In [ ]:
def _indicators(self,radius):
        """
        indicator function using 0/1 to represent if the euclidean distance to the
        central point is less than the chosen radius
        Parameters
        ----------
        radius: :obj: `float`
        """
        grid_width = self.grid_width
        tmp = np.arange(start=1,stop=grid_width+1, step=1) #np array: [1, 3, 5, ..., 65]
        tmp_coords = list()
        for element in itertools.product(tmp,tmp,tmp):
            tmp_coords.append(list(element))
        tmp_coords = np.array(tmp_coords) #shape:(35937, 3)

        length = grid_width**3 #35937 by default
        center_coord = np.array([[grid_width,grid_width,grid_width]]*length) #shape:(35937, 3)
        dist = np.linalg.norm(tmp_coords - center_coord, axis=1).reshape((grid_width,grid_width,grid_width))
            
        # use 0/1 to indicate if the euclidean distrance is less than the specified radius
        dist[dist < radius] = 1
        dist[dist > radius] = 0

        return dist

In [199]:
a = np.load('n_total120n_valid5_general_activation.npy')

In [200]:
a

array([[1061., 1061., 1298., 1194., 1240., 1240., 1195., 1005.],
       [1102., 1102.,  848., 1380.,  791.,  791., 1285.,  870.]])